# Utilizing Mobile Positioning Data (MPD) to Measure SDGs Indicator 17.8.1: Proportion of Individuals Using the Internet

## Summary

`SDG Indicator 17.8.1` aims to assess the proportion of individuals using the Internet, specifically those who have accessed it within the last three months. The Internet, being a global computer network that facilitates various communication services like the World Wide Web, email, news, and entertainment, can be accessed through different devices such as computers, mobile phones, tablets, PDAs, gaming consoles, and digital TVs. Connectivity may be established through fixed or mobile networks.

Leveraging mobile phone data offers a valuable avenue for estimating and enhancing the measurement of this indicator. The utilization of mobile phone data can significantly contribute to the accurate and detailed assessment of SDG Indicator 17.8.1 by providing real-time and granular information on Internet usage. However, it is essential to approach data privacy, data quality, and the need for complementary data sources with careful consideration to ensure the reliability and effectiveness of the measurement process.

When feasible, the indicator can be further analyzed by various breakdowns and disaggregation criteria, such as regional distinctions (urban and rural areas), gender, age groups, etc. The International Telecommunication Union (ITU) collects data from countries encompassing these breakdowns to enhance the comprehensiveness of the indicator's assessment.

## Calculation of Indicator

### add highlight for hashing consistency from MNO

To obtain geographical coordinates and facilitate data aggregation, it is essential to combine Mobile Positioning Data (MPD) with cell data obtained from Mobile Network Operators (MNOs) or other publicly available sources. The cell data includes approximate geographical coordinates and the corresponding technology used. Each cell's location, categorized as the subscriber's home location, is then aggregated at the desired geographical level, such as LAU 2, and separated based on the technology used for internet connectivity.

The following assumptions are made regarding subscriber internet usage:

- If a subscriber has no mobile internet traffic (IPDR), it is assumed that they do not have internet access.
- If a subscriber only has mobile internet traffic in 2G cells (without any events in 3G or 4G), it is presumed that they solely use the internet through 2G.
- If a subscriber has any mobile internet traffic in 3G cells (potentially in 2G as well, but not in 4G), it is inferred that they use the internet through 3G.
- If a subscriber has any mobile internet traffic in 4G cells (and may also have usage in 2G and 3G), it is deduced that they utilize 4G for internet connectivity.

The resulting dataset contains the following fields:

- Level of aggregation, which can be demographic variables like gender and age group, administrative area, or a combination of them.
- total_home_count: number of persons with detected homes.
- no_data_home_count: number of subscribers who have no mobile internet traffic and have detected homes.
- 2G_home_count (2G): number of subscribers who have 2G mobile data traffic and a detected home.
- 3G_home_count (3G): number of subscribers who have 3G mobile data traffic and a detected home.
- 4G_home_count (4G): number of subscribers who have 4G data traffic and a detected home.

The resulting table will show the proportion of subscribers using the internet by technology in each group. To extrapolate this information to the rest of the population, additional data on mobile access is required. Since the MNO was also asked to indicate the cell type (2G, 3G, or 4G), the geographical distribution should be checked by cell type through visual analysis.

## Steps for Measurement
1. **Define the Scope of Aggregation**: Determine the geographic level (global, regional, national, or local) and specify the partnerships or initiatives to be considered for measurement.
2. **Identify Data Sources**: Join information from anchoring process with another data source like Customer relationship management (CRM) data that containing information about subscribers demography, and other data relevant data sources.
3. **Calculate the Indicator**: Use the formula below to calculate the indicator:
   ```
   Indicator 17.8.1 = (Number of individuals using the internet / Total number of individuals in the target population) x 100%
   ```
4. **Data Validation**: Validate the accuracy and reliability of the proxy indicator from mobile data by comparing it with other source including conventional survey to ensure its quality and coherence.

In [ ]:
# !pip install pygwalker -q

In [ ]:
# Import the sys module to adjust the Python system path
import sys

# Import the os module to interact with the operating system
import os
os.environ["JAVA_HOME"] = "/home/ec2-user/anaconda3/envs/spark"

# Append a path to the system path so that Python can import modules from the specified directory (relative path)
sys.path.append('../')

# Import the CONF, QA_summary and QA_action_plan variables from the script.QA module
from script.conf import *

In [ ]:
# Try to create a new directory at the path QA_PATH using os.mkdir()
# If the directory already exists, print a message stating that it does
try:
    os.mkdir(INDICATOR_PATH)
    print("Create new folder {}".format(INDICATOR_PATH))
except FileExistsError:
    print("Folder {} already exists".format(INDICATOR_PATH))

In [ ]:
# Import Pandas module
import pandas as pd

# Import the PySpark module
import pyspark

# Import the Seaborn module
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
sns.set_color_codes("pastel")

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import year, month, dayofmonth, substring, col, floor, expr, when, mean, count
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType

# Import the SparkSession from PySpark
from pyspark.sql import SparkSession

# Create a SparkSession with the specified configuration
spark = SparkSession.builder\
        .master("local[{}]".format(CORE))\
        .appName("02.ITU.PySpark-Indicator")\
        .config('spark.sql.execution.arrow.pyspark.enabled', 'true')\
        .config('spark.eventLog.gcMetrics.youngGenerationGarbageCollectors', 'true')\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Print the spark object which contains the SparkSession
spark

In [ ]:
# Use this code below to connect your remote spark cluster instead.

# # Import the PySpark module
# import pyspark
# from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType

# import pyspark.sql.functions as f
# from pyspark.sql.functions import year, month, dayofmonth, substring, col, to_date
# from pyspark.sql.window import Window

# # Import the SparkSession from PySpark
# from pyspark.sql import SparkSession

# CLUSTER_URL = "spark://master"
# PORT = "7077"

# # Create a SparkSession with the specified configuration remote server
# spark = SparkSession.builder\
#         .master("{}:{}".format(CLUSTER_URL,PORT))\
#         .appName("01.ITU.PySpark-Raw")\
#         .getOrCreate()

# # Print the spark object which contains the SparkSession
# spark

In [ ]:
# Set the path to the cell administrative location file. In this example, we will use data generated from QA step
# This is mandatory file that will be needed for indicator aggregation
path_cell = QA_PATH+'5_cell_administrative_location.csv'

# Read the cell administrative location file using Spark
cell = spark.read.format('csv')\
    .options(delimiter='\t',inferSchema="true")\
    .option('header',True)\
    .load(path_cell)

# Rename the 'cell_id' column to 'anchor_cell_id'
cell = cell.withColumn('cell_id',col('cell_id').cast(IntegerType()))
cell = cell.withColumnRenamed('cell_id','anchor_cell_id')
cell.show(5)

In [ ]:
# Set the path to the zone file
path_zone = BASE_PATH+INDICATOR_ZONE_PATH

if os.path.exists(path=path_zone):
    ZONE_FLAG = True
    # Read the zone file using Spark
    zone = spark.read.format('csv')\
        .options(delimiter=',',inferSchema="true")\
        .option('header',True)\
        .load(path_zone)

    # Rename the 'cell_id' column to 'anchor_cell_id'
    zone = zone.withColumnRenamed('cell_id','anchor_cell_id')
    zone.show(5)
else:
    ZONE_FLAG = False
    print("Information about ZONAL CLASSIFICATION (Urban, Rural) of the cell location is missing.")

In [ ]:
# Set the path to the CRM file
path_crm = BASE_PATH+INDICATOR_CRM_PATH

if os.path.exists(path=path_crm):
    CRM_FLAG = True
    # Read the CRM file using Spark
    crm = spark.read.format('csv')\
        .options(delimiter=',',inferSchema="true")\
        .option('header',True)\
        .load(path_crm)

    # Create a new column 'age_group' by concatenating the age range
    crm = crm.withColumn('age_group',expr("concat(floor(age / 10) * 10, '-', floor(age / 10) * 10 + 10)"))

    # Show the contents of the CRM DataFrame
    crm.show()
else:
    CRM_FLAG = False
    print("Information about CUSTOMER RELATIONSHIP MANAGEMENT (Gender, Age Group) of the subscribers is missing.")

In [ ]:
# Set the path to the user anchor final file
path = ANCHOR_PATH+'user_anchor_final.csv'

# Read the user anchor final file using Spark, specifying the file format as CSV and setting the delimiter as tab. It also infers the schema from the data and treats the first row as the header.
df = spark.read.format('csv')\
    .options(delimiter='\t',inferSchema="true")\
    .option('header',True)\
    .load(path)

# Filter out rows where the 'anchor_cell_id' column is null
df = df.filter(col('anchor_cell_id').isNotNull())

# Join the df DataFrame with the cell DataFrame on the 'anchor_cell_id' column, using a left join
df = df.join(cell,['anchor_cell_id'],how='left')

if ZONE_FLAG:
    # Join the df DataFrame with the zone DataFrame on the 'anchor_cell_id' column, using a left join
    df = df.join(zone,['anchor_cell_id'],how='left')


if CRM_FLAG:
    # Join the df DataFrame with the crm DataFrame on the 'msisdn' column, using a left join
    df = df.join(crm,['msisdn'],how='left')

df.cache()

# Show the contents of the df DataFrame
df.show()

In [ ]:
df.printSchema()

The code starts by defining a variable `tech_col` with the value 'IPDR_highest_service'.

Then, it uses the `groupBy()` function on a DataFrame `df` to group the data by the 'municipality' column.

After that, it calls the `agg()` function to perform multiple aggregation operations on the grouped data. Each aggregation operation is specified as a separate argument to the `agg()` function.

Here are the aggregation operations being performed:

1. `count(col('msisdn')).alias('total_home_count')`: This counts the number of rows in each group and assigns the alias 'total_home_count' to the resulting column.

2. `count(when(col('internet_user')==False,col('msisdn'))).alias('total_no_internet')`: This counts the number of rows where the 'internet_user' column is False, and assigns the alias 'total_no_internet' to the resulting column.

3. `count(when((col('internet_user')==True) & (col(tech_col)=='2G'),col('msisdn'))).alias('total_internet_2G')`: This counts the number of rows where the 'internet_user' column is True and the 'IPDR_highest_service' column is '2G', and assigns the alias 'total_internet_2G' to the resulting column.

4. `count(when((col('internet_user')==True) & (col(tech_col)=='3G'),col('msisdn'))).alias('total_internet_3G')`: This counts the number of rows where the 'internet_user' column is True and the 'IPDR_highest_service' column is '3G', and assigns the alias 'total_internet_3G' to the resulting column.

5. `count(when((col('internet_user')==True) & (col(tech_col)=='4G'),col('msisdn'))).alias('total_internet_4G')`: This counts the number of rows where the 'internet_user' column is True and the 'IPDR_highest_service' column is '4G', and assigns the alias 'total_internet_4G' to the resulting column.

In [ ]:
# Define the column name to be used in the aggregation
tech_col = 'IPDR_highest_service'

# Group the dataframe 'df' by the 'municipality' column and perform aggregations
ind_adm = df.groupBy(['municipality','geomatch']).agg(
        count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
        count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
        count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
        count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
        count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
        count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
        count(when((col('internet_user')==True) & (col(tech_col)=='5G'), col('msisdn'))).alias('total_internet_5G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '5G', and alias the column as 'total_internet_5G'
    )\
    .orderBy(['total_internet_ALL'],ascending=False).toPandas()

ind_adm['perc_no_internet'] = (ind_adm['total_no_internet'] / ind_adm['total_home_count'].replace({0: pd.NA}))
ind_adm['perc_internet'] = (ind_adm['total_internet_ALL'] / ind_adm['total_home_count'].replace({0: pd.NA}))
ind_adm['perc_internet_2G'] = (ind_adm['total_internet_2G'] / ind_adm['total_home_count'].replace({0: pd.NA}))
ind_adm['perc_internet_3G'] = (ind_adm['total_internet_3G'] / ind_adm['total_home_count'].replace({0: pd.NA}))
ind_adm['perc_internet_4G'] = (ind_adm['total_internet_4G'] / ind_adm['total_home_count'].replace({0: pd.NA}))
ind_adm['perc_internet_5G'] = (ind_adm['total_internet_5G'] / ind_adm['total_home_count'].replace({0: pd.NA}))

ind_adm


In [ ]:
ind_adm.to_csv(INDICATOR_PATH+"ind_adm.csv",sep="\t",index=False)

# Visualizations

### Bar chart of Internet use by municipality

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 14))  # wider=10, taller=14 (increase this height to make it taller)
ax = sns.barplot(
    data=ind_adm,
    y='municipality',
    x='perc_internet',
    hue='municipality',
    orient='h'
)

# Make axis labels and tick labels smaller
ax.set_xlabel('perc_internet', fontsize=10)
ax.set_ylabel('municipality', fontsize=10)
ax.tick_params(axis='both', labelsize=8)

# Optional: shrink legend text too
leg = ax.get_legend()
if leg is not None:
    leg.set_title(None)
    for t in leg.texts:
        t.set_fontsize(8)

plt.tight_layout()
plt.show()

In [ ]:
# Importing geopandas and shapely.geometry libraries 
import json, geopandas as gpd
from shapely.geometry import Point, shape
import folium

# Providing the path to the GeoJSON file containing administrative boundaries. 

# Option 1 of reading GEOJSON_FILE - requires Fiona package
#adm_boundaries = gpd.read_file(GEOJSON_FILE)

# Option 2 of reading GEOJSON_FILE
with open(GEOJSON_FILE, "r", encoding="utf-8") as f:
    gj = json.load(f)

features = gj["features"]
geoms = [shape(feat["geometry"]) for feat in features]
props = [feat.get("properties", {}) for feat in features]

adm_boundaries = gpd.GeoDataFrame(props, geometry=geoms, crs="EPSG:4326")
adm_boundaries = adm_boundaries.rename(columns={MUNICIPALITY_FIELD_NAME:'municipality'})
adm_boundaries = adm_boundaries.rename(columns={MUNICIPALITY_MATCH_NAME:'geomatch'})

adm_boundaries = adm_boundaries.merge(ind_adm)

# Displaying the resulting geopandas dataframe `adm_boundaries`.
adm_boundaries.head()

### Map of Internet use by municipality

In [ ]:
# visualize the indicator using interactive maps
# Create a folium map object and set the initial location and zoom level
m = folium.Map(location=[adm_boundaries.bounds.iloc[0]['miny'],adm_boundaries.bounds.iloc[0]['minx']], zoom_start=10, tiles="CartoDB positron")

# Add a tile layer to the map
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

# Add a choropleth layer to represent the home location data
# folium.Choropleth(
#     name="home_loc",
#     geo_data=adm_boundaries,
#     data=ind_adm,
#     columns=["geomatch","total_home_count"],
#     key_on='feature.properties.geomatch',
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     highlight=True,
#     show=False,
#     legend_name="Population Density",
# ).add_to(m)

# Add another choropleth layer to represent the internet usage data
folium.Choropleth(
    name="internet_usage",
    geo_data=adm_boundaries,
    data=ind_adm,
    columns=["geomatch","perc_internet"],
    key_on='feature.properties.geomatch',
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    show=True,
    legend_name="Internet Usage (%)",
).add_to(m)

# Define a style function for the GeoJson layer
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

# Define a highlight function for the GeoJson layer
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

# Create a GeoJson layer with style and highlight functions
NIL = folium.features.GeoJson(
    adm_boundaries,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['municipality','total_home_count','perc_internet'],
        aliases=['Municipality: ','Population Density: ','Internet Usage %: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px; font-weight: bold;") 
    )
)

# Add the GeoJson layer to the map
m.add_child(NIL)

# Keep the GeoJson layer in front of other layers on the map
m.keep_in_front(NIL)


In [ ]:
# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save(INDICATOR_PATH+"ITU_internet_usage.html")

# Display the map
m

### Map showing Internet use by technology 

In [ ]:
# Make sure you’ve already run your aggregation above to produce ind_adm

# 1️⃣  Create a long (melted) version of the ind_adm dataset
ind_adm_long = ind_adm.melt(
    id_vars=['geomatch', 'municipality', 'perc_internet'],   # <— add perc_internet here
    value_vars=['perc_no_internet', 'perc_internet_2G', 'perc_internet_3G', 'perc_internet_4G', 'perc_internet_5G'],
    var_name='category',
    value_name='value'
)

# 2️⃣  Clean category labels
ind_adm_long['category'] = ind_adm_long['category'].replace({
    'perc_no_internet': 'No Internet',
    'perc_internet_2G': '2G',
    'perc_internet_3G': '3G',
    'perc_internet_4G': '4G',
    'perc_internet_5G': '5G'
})

# 3️⃣  Replace NaN with 0 for plotting safety
ind_adm_long['value'] = ind_adm_long['value'].fillna(0)

# 4️⃣  Also add percentage columns in percent form for tooltips or display
ind_adm_long['perc_internet_pct'] = (ind_adm_long['perc_internet'] * 100).round(1)
ind_adm_long['value_pct'] = (ind_adm_long['value'] * 100).round(1)

# 5️⃣  Optional: quick check
print(ind_adm_long.head(10))

In [ ]:
import io, base64
import pandas as pd

# -----------------------------------------------------
# 1️⃣ Prepare the data
# -----------------------------------------------------
# ind_adm_long already created as:
# geomatch | municipality | perc_internet | category | value

# Replace NaN safely
ind_adm_long['value'] = ind_adm_long['value'].fillna(0)
ind_adm_long['perc_internet'] = ind_adm_long['perc_internet'].fillna(0)

# Round for display
ind_adm_long['value_pct'] = (ind_adm_long['value'] * 100).round(1)
ind_adm_long['perc_internet_pct'] = (ind_adm_long['perc_internet'] * 100).round(1)

# -----------------------------------------------------
# 2️⃣ Base Map (no choropleth fill)
# -----------------------------------------------------
m = folium.Map(
    location=[adm_boundaries.bounds.iloc[0]['miny'], adm_boundaries.bounds.iloc[0]['minx']],
    zoom_start=8,
    tiles="CartoDB positron"
)
folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)

# -----------------------------------------------------
# 3️⃣ GeoJSON layer with tooltip
# -----------------------------------------------------
# Merge ind_adm (not ind_adm_long) to adm_boundaries for tooltips
adm_boundaries_tooltip = adm_boundaries

# Convert to percentages
for col in ['perc_no_internet','perc_internet_2G','perc_internet_3G','perc_internet_4G','perc_internet_5G','perc_internet']:
    adm_boundaries_tooltip[col] = (adm_boundaries_tooltip[col].fillna(0) * 100).round(1)

# Define tooltip style
style_function = lambda x: {'fillColor': 'transparent', 'color':'#000000', 'fillOpacity':0, 'weight':0.7}
highlight_function = lambda x: {'fillColor':'#000000', 'color':'#000000', 'fillOpacity':0.2, 'weight':0.5}

# Tooltip content
NIL = folium.GeoJson(
    adm_boundaries_tooltip,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['municipality','total_home_count',
                'perc_no_internet','perc_internet_2G','perc_internet_3G','perc_internet_4G','perc_internet_5G','perc_internet'],
        aliases=['Municipality: ','Population: ',
                 'No Internet (%): ','2G (%): ','3G (%): ','4G (%): ','5G (%): ','Total Internet (%): '],
        localize=True,
        sticky=False,
        labels=True,
        style=("background-color: white; color: #333333; font-family: arial; "
               "font-size: 12px; padding: 10px; font-weight: bold;")
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# -----------------------------------------------------
# 4️⃣ Add Pie Charts per Municipality
# -----------------------------------------------------
color_map = {
    'No Internet': '#aaaaaa',
    '2G': '#ffcc00',
    '3G': '#6699ff',
    '4G': '#33cc33',
    '5G': '#ea9999'
}

for geomatch, subset in ind_adm_long.groupby('geomatch'):
    geom = adm_boundaries.loc[adm_boundaries['geomatch'] == geomatch, 'geometry']
    if geom.empty:
        continue
    centroid = geom.values[0].centroid
    lat, lon = centroid.y, centroid.x

    if subset['value'].sum() == 0:
        continue

    labels = subset['category']
    sizes = subset['value']
    colors = [color_map[c] for c in labels]

    # Create pie chart image
    fig, ax = plt.subplots(figsize=(1.2, 1.2))
    ax.pie(sizes, labels=None, startangle=90, colors=colors)
    plt.axis('equal')

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', transparent=True)
    plt.close(fig)
    b64 = base64.b64encode(buf.getvalue()).decode()

    # Add the pie chart to the map
    html = f'<img src="data:image/png;base64,{b64}" width="55" height="55">'
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(html=html)
    ).add_to(m)

# -----------------------------------------------------
# 5️⃣ Add Legend
# -----------------------------------------------------
legend_html = """
<div style="
    position: fixed; 
    bottom: 20px; right: 20px; 
    width: 160px; 
    background-color: white; 
    border:2px solid grey; 
    z-index:9999; 
    font-size:12px;
    border-radius:8px;
    padding: 8px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
<b>Internet Type</b><br>
<div style="display:flex;align-items:center;"><div style="width:15px;height:15px;background:#aaaaaa;margin-right:6px;"></div>No Internet</div>
<div style="display:flex;align-items:center;"><div style="width:15px;height:15px;background:#ffcc00;margin-right:6px;"></div>2G</div>
<div style="display:flex;align-items:center;"><div style="width:15px;height:15px;background:#6699ff;margin-right:6px;"></div>3G</div>
<div style="display:flex;align-items:center;"><div style="width:15px;height:15px;background:#33cc33;margin-right:6px;"></div>4G</div>
<div style="display:flex;align-items:center;"><div style="width:15px;height:15px;background:#ea9999;margin-right:6px;"></div>5G</div>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# -----------------------------------------------------
# 6️⃣ Display the Map
# -----------------------------------------------------

# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save(INDICATOR_PATH+"ITU_internet_usage_by_technology.html")

# Display the map
m

### Map showing the zone classification 

In [ ]:
from pyspark.sql.functions import col, when, count

# Check if ZONE_FLAG is True
if ZONE_FLAG:
    # If ZONE_FLAG is True, group the dataframe 'df' by 'zone_classification' and perform aggregations
    ind_zone = df.groupBy(['zone_classification']).agg(
        count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
        count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
        count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
        count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
        count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
        count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
    ).orderBy(['zone_classification']).toPandas()
    
    ind_zone['perc_no_internet'] = (ind_zone['total_no_internet'] / ind_zone['total_home_count'].replace({0: pd.NA}))
    ind_zone['perc_internet'] = (ind_zone['total_internet_ALL'] / ind_zone['total_home_count'].replace({0: pd.NA}))
    ind_zone['perc_internet_2G'] = (ind_zone['total_internet_2G'] / ind_zone['total_home_count'].replace({0: pd.NA}))
    ind_zone['perc_internet_3G'] = (ind_zone['total_internet_3G'] / ind_zone['total_home_count'].replace({0: pd.NA}))
    ind_zone['perc_internet_4G'] = (ind_zone['total_internet_4G'] / ind_zone['total_home_count'].replace({0: pd.NA}))
    
    ind_zone.to_csv(INDICATOR_PATH+"ind_zone.csv",sep="\t",index=False)

    # Group the dataframe 'df' by both 'municipality' and 'zone_classification', and perform aggregations
    ind_adm_zone = df.groupBy(['municipality','zone_classification']).agg(
        count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
        count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
        count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
        count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
        count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
        count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
    ).orderBy(['municipality','zone_classification']).toPandas()
    
    ind_adm_zone['perc_no_internet'] = (ind_adm_zone['total_no_internet'] / ind_adm_zone['total_home_count'].replace({0: pd.NA}))
    ind_adm_zone['perc_internet'] = (ind_adm_zone['total_internet_ALL'] / ind_adm_zone['total_home_count'].replace({0: pd.NA}))
    ind_adm_zone['perc_internet_2G'] = (ind_adm_zone['total_internet_2G'] / ind_adm_zone['total_home_count'].replace({0: pd.NA}))
    ind_adm_zone['perc_internet_3G'] = (ind_adm_zone['total_internet_3G'] / ind_adm_zone['total_home_count'].replace({0: pd.NA}))
    ind_adm_zone['perc_internet_4G'] = (ind_adm_zone['total_internet_4G'] / ind_adm_zone['total_home_count'].replace({0: pd.NA}))
    
    ind_adm_zone.to_csv(INDICATOR_PATH+"ind_adm_zone.csv",sep="\t",index=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 16))  # wider=10, taller=14 (increase this height to make it taller)
ax = sns.barplot(
    data=ind_adm_zone,
    y='municipality',
    x='perc_internet',
    hue='zone_classification',
    orient='h'
)

# Make axis labels and tick labels smaller
ax.set_xlabel('perc_internet', fontsize=10)
ax.set_ylabel('municipality', fontsize=10)
ax.tick_params(axis='both', labelsize=8)

# Optional: shrink legend text too
leg = ax.get_legend()
if leg is not None:
    leg.set_title(None)
    for t in leg.texts:
        t.set_fontsize(8)

plt.tight_layout()
plt.show()

In [ ]:
# Check if CRM_FLAG is True
if CRM_FLAG:
    # If CRM_FLAG is True, group the dataframe 'df' by 'age_group' and perform aggregations
    ind_age = df.groupBy(['age_group']).agg(
        count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
        count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
        count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
        count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
        count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
        count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
    ).orderBy(['age_group']).toPandas()
    
    ind_age['perc_no_internet'] = (ind_age['total_no_internet'] / ind_age['total_home_count'].replace({0: pd.NA}))
    ind_age['perc_internet'] = (ind_age['total_internet_ALL'] / ind_age['total_home_count'].replace({0: pd.NA}))    
    ind_age['perc_internet_2G'] = (ind_age['total_internet_2G'] / ind_age['total_home_count'].replace({0: pd.NA}))
    ind_age['perc_internet_3G'] = (ind_age['total_internet_3G'] / ind_age['total_home_count'].replace({0: pd.NA}))
    ind_age['perc_internet_4G'] = (ind_age['total_internet_4G'] / ind_age['total_home_count'].replace({0: pd.NA}))
        
    # Convert the resulting dataframe to a Pandas DataFrame
    # Save the Pandas DataFrame to a CSV file with the name 'ind_age.csv' at the specified path 'INDICATOR_PATH', using tab as the separator and excluding the index column
    ind_age.to_csv(INDICATOR_PATH+"ind_age.csv", sep="\t", index=False)

    # Group the dataframe 'df' by 'gender' and perform aggregations
    ind_gen = df.groupBy(['gender']).agg(
        count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
        count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
        count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
        count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
        count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
        count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
    ).orderBy(['gender']).toPandas()
    
    ind_gen['perc_no_internet'] = (ind_gen['total_no_internet'] / ind_gen['total_home_count'].replace({0: pd.NA}))
    ind_gen['perc_internet'] = (ind_gen['total_internet_ALL'] / ind_gen['total_home_count'].replace({0: pd.NA}))    
    ind_gen['perc_internet_2G'] = (ind_gen['total_internet_2G'] / ind_gen['total_home_count'].replace({0: pd.NA}))
    ind_gen['perc_internet_3G'] = (ind_gen['total_internet_3G'] / ind_gen['total_home_count'].replace({0: pd.NA}))
    ind_gen['perc_internet_4G'] = (ind_gen['total_internet_4G'] / ind_gen['total_home_count'].replace({0: pd.NA}))
       
    # Convert the resulting dataframe to a Pandas DataFrame
    # Save the Pandas DataFrame to a CSV file with the name 'ind_gen.csv' at the specified path 'INDICATOR_PATH', using tab as the separator and excluding the index column
    ind_gen.to_csv(INDICATOR_PATH+"ind_gen.csv", sep="\t", index=False)
    
    # Group the dataframe 'df' by 'gender' and 'age_group' columns and perform aggregations
    ind_gen_age = df.groupBy(['gender', 'age_group']).agg(
        count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
        count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
        count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
        count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
        count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
        count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
    ).orderBy(['age_group', 'gender']).toPandas()
    
    ind_gen_age['perc_no_internet'] = (ind_gen_age['total_no_internet'] / ind_gen_age['total_home_count'].replace({0: pd.NA}))
    ind_gen_age['perc_internet'] = (ind_gen_age['total_internet_ALL'] / ind_gen_age['total_home_count'].replace({0: pd.NA}))
    ind_gen_age['perc_internet_2G'] = (ind_gen_age['total_internet_2G'] / ind_gen_age['total_home_count'].replace({0: pd.NA}))
    ind_gen_age['perc_internet_3G'] = (ind_gen_age['total_internet_3G'] / ind_gen_age['total_home_count'].replace({0: pd.NA}))
    ind_gen_age['perc_internet_4G'] = (ind_gen_age['total_internet_4G'] / ind_gen_age['total_home_count'].replace({0: pd.NA}))
    
    # Convert the resulting dataframe to a Pandas DataFrame
    # Save the Pandas DataFrame to a CSV file named 'ind_gen_age.csv' at the specified path 'INDICATOR_PATH', using tab as the separator and excluding the index column
    ind_gen_age.to_csv(INDICATOR_PATH+"ind_gen_age.csv", sep="\t", index=False)
 
    # Group the dataframe 'df' by 'municipality', 'gender', and 'age_group' columns and perform aggregations
    # ind_adm_gen_age = df.groupBy(['municipality', 'gender', 'age_group']).agg(
    #     count(col('msisdn')).alias('total_home_count'),  # Count the number of 'msisdn' values and alias the column as 'total_home_count'
    #     count(when(col('internet_user')==False, col('msisdn'))).alias('total_no_internet'),  # Count the number of 'msisdn' values where 'internet_user' is False and alias the column as 'total_no_internet'
    #     count(when(col('internet_user')==True, col('msisdn'))).alias('total_internet_ALL'),  # Count the number of 'msisdn' values where 'internet_user' is True, and alias the column as 'total_internet_ALL'
    #     count(when((col('internet_user')==True) & (col(tech_col)=='2G'), col('msisdn'))).alias('total_internet_2G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '2G', and alias the column as 'total_internet_2G'
    #     count(when((col('internet_user')==True) & (col(tech_col)=='3G'), col('msisdn'))).alias('total_internet_3G'),  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '3G', and alias the column as 'total_internet_3G'
    #     count(when((col('internet_user')==True) & (col(tech_col)=='4G'), col('msisdn'))).alias('total_internet_4G')  # Count the number of 'msisdn' values where 'internet_user' is True and 'tech_col' is '4G', and alias the column as 'total_internet_4G'
    # ).orderBy(['municipality', 'age_group', 'gender']).toPandas()
    # # Convert the resulting dataframe to a Pandas DataFrame
    # # Save the Pandas DataFrame to a CSV file named 'ind_adm_gen_age.csv' at the specified path 'INDICATOR_PATH', using tab as the separator and excluding the index column
    # ind_adm_gen_age.to_csv(INDICATOR_PATH+"ind_adm_gen_age.csv", sep="\t", index=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  # wider=10, taller=14 (increase this height to make it taller)
ax = sns.barplot(
    data=ind_gen_age,
    y='age_group',
    x='perc_internet',
    hue='gender',
    orient='h'
)

# Make axis labels and tick labels smaller
ax.set_xlabel('perc_internet', fontsize=10)
ax.set_ylabel('age_group', fontsize=10)
ax.tick_params(axis='both', labelsize=8)

# Optional: shrink legend text too
leg = ax.get_legend()
if leg is not None:
    leg.set_title(None)
    for t in leg.texts:
        t.set_fontsize(8)

plt.tight_layout()
plt.show()